In [ ]:
import matplotlib.pyplot as plt
from Genetic.Representation import Function
from Genetic.NeuroEvolution import NeuroEvolution
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mse
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
from Model.model import SurvModel, FlchainModel, NwtcoModel, SupportModel
from Data.dataset import Flchain, Support, Nwtco
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import f_oneway, ttest_rel

from Utils.utils import Experiment, FunctionHelper, StatsHelper
from itertools import combinations

In [ ]:
surv_reg1_func = [1, 9, 9, 1, 5, 12, 7, 9, 7]
surv_reg2_func = [1, 9, 12, 1, 5, 12, 7, 9, 7]

surv_reglog_func = [1, 9, 17, 1, 1, 12, 7, 9, 7]
surv_reglog_trunc_func = [1, 9, 29, 1, 2, 12, 7, 17, 31]
surv_reglog_notrunc_func = [1, 9, 9, 1, 2, 12, 7, 17, 31]

surv_reg1_f = FunctionHelper(surv_reg1_func)
surv_reg2_f = FunctionHelper(surv_reg2_func)
surv_reglog_f = FunctionHelper(surv_reglog_func)
surv_reglog_trunc_f = FunctionHelper(surv_reglog_trunc_func)
surv_reglog_notrunc_f = FunctionHelper(surv_reglog_notrunc_func)

surv_reg1_f.plot_surv_loss()
surv_reg2_f.plot_surv_loss()
surv_reglog_f.plot_surv_loss(y_min=-2, y_max=2, subtitle='SurvRegLog')
surv_reglog_notrunc_f.plot_surv_loss(y_min=-2, y_max=2, subtitle='SurvRegLog_Notrunc')
surv_reglog_trunc_f.plot_surv_loss(y_min=-2, y_max=2, subtitle='SurvRegLog_trunc')


In [ ]:
model_class = FlchainModel
ds = Flchain('Data/flchain.csv')

In [ ]:
(x_train, ye_train, y_train, e_train,
 x_val, ye_val, y_val, e_val,
 x_test, ye_test, y_test, e_test) = ds.get_train_val_test_from_splits(test_id=0, val_id=1)
xy = (x_train, ye_train), (x_val, ye_val), (x_test, ye_test)

In [ ]:
exp_reglog_trunc = Experiment(model_class=model_class, 
                  ds=ds, 
                  loss_f=surv_reglog_trunc_f.f, 
                  step_f=None, 
                  use_clb=False, 
                  batch_size=128, 
                  epochs=500, 
                  patience=100, 
                  verbose=False)
exp_reglog_trunc.get_results_cv()

In [ ]:
exp_reglog_notrunc = Experiment(model_class=model_class, 
                  ds=ds, 
                  loss_f=surv_reglog_notrunc_f.f, 
                  step_f=None, 
                  use_clb=False, 
                  batch_size=128, 
                  epochs=500, 
                  patience=100, 
                  verbose=False)
exp_reglog_notrunc.get_results_cv()

In [ ]:
StatsHelper.plot_box(folds1=exp_reglog_trunc.folds_scores, name1='softplus_truncated', folds2=exp_reglog_notrunc.folds_scores, name2='softplus')